# **Respiratory Sound Classification** Based on BiGRU-Attention Network with XGBoost


### Importing Packages

In [ ]:
!pip install pyACA

     |████████████████████████████████| 57 kB 2.6 MB/s 


In [ ]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import pyACA
import sklearn
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import train_test_split as tts
from sklearn import metrics
from sklearn.utils import shuffle
from xgboost import XGBClassifier as XGB
import librosa
import librosa.display
import IPython.display as ipd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. **Feature Extraction**
We will be using following features for the classification purpose :
* Short-term Average Energy
* Short-term Average Zero-crossing Rate
* Spectral Centroid
* Spectral Slope
* Spectral Contrast
* RMS Energy
* Chromaticity
* MFCC

In [ ]:
dir_path = '/content/drive/MyDrive/ML_DC_Project/dc_dataset/'
csv_path = '/content/drive/MyDrive/ML_DC_Project/mldata.csv'
np_dir = '/content/drive/MyDrive/ML_DC_Project/np_arrays/'
final_dataset = '/content/drive/MyDrive/ML_DC_Project/final_dataset/'

f_start = 101
f_end = 639

In [ ]:
FRAME_LENGTH = 1024
HOP_LENGTH = 512

def SpectralSlope(data, sr, hop_length):
    ss = []
    for i in range(0,len(data),512):
        slice = data[i:i+512]
        s = pyACA.FeatureSpectralSlope(slice, f_s=sr)
        ss.append(s*512)
    return np.array(ss)

def grad(arr):
    arr = np.array(arr)
    grad_arr = []
    for i in range(arr.shape[0]):
        grad_arr.append(np.gradient(arr[i]))
    return np.array(grad_arr)

def extract_features(file_path: str, frame_length, hop_length):

    audio_signal, sr = librosa.load(file_path)

    stft = np.abs(librosa.stft(audio_signal))
    mfccs_features = librosa.feature.mfcc(audio_signal, sr=sr, n_mfcc=12)
    # mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    mfccs_grad = grad(mfccs_features)
    all_mfccs = np.concatenate((mfccs_features,mfccs_grad))
    # chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    chroma = librosa.feature.chroma_stft(S=stft, sr=sr)
    # contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sr)
    zcr = frame_length*(librosa.feature.zero_crossing_rate(audio_signal, frame_length=frame_length, hop_length=hop_length)[0])
    rms = librosa.feature.rms(audio_signal, frame_length=frame_length, hop_length=hop_length)[0]
    spectral_centroid = librosa.feature.spectral_centroid(audio_signal, sr=sr, n_fft=frame_length, hop_length=hop_length)[0]
    spectral_slope = SpectralSlope(audio_signal, sr, hop_length)

    return [zcr, rms, spectral_centroid, all_mfccs, chroma, contrast, spectral_slope]


Note : 3 cells below are not to be run again as the dataset is already stored in .npz files

In [ ]:
# FRAME_LENGTH = 1024
# HOP_LENGTH = 512

# for patient in tqdm(range(f_start,f_end+1), 'Extraction Progress '):
#   file_name = os.path.join(os.path.abspath(dir_path)+'/'+str(patient)+'.wav')
#   extracted_arrs = extract_features(file_name,FRAME_LENGTH,HOP_LENGTH)
#   np.savez(np_dir+str(patient)+'.npz',zcr = extracted_arrs[0],rms = extracted_arrs[1],spectral_centroid = extracted_arrs[2],\
#            mfcc = extracted_arrs[3], chroma = extracted_arrs[4],spectral_contrast = extracted_arrs[5], spectral_slope=extracted_arrs[6])

Extraction Progress : 100%|██████████| 539/539 [10:59<00:00,  1.22s/it]


In [ ]:
# data_3d = []

# for patient in range(f_start,f_end+1):
#     arrs = np.load(np_dir+str(patient)+'.npz')
#     final = []
#     for i in arrs:
#         f_arr = None
#         if patient==f_start:
#             print(i,arrs[i].shape)
#         if arrs[i].shape[0]<100:
#             for ar in arrs[i]:
#                 f_arr = ar
#                 if f_arr.shape[0]<862:
#                     new_f = np.zeros(862)
#                     for ii in range(f_arr.shape[0]):
#                         new_f[ii] = f_arr[ii]
#                     final.append(new_f)
#                 else:
#                     final.append(f_arr[:862])
#         else:
#             f_arr = arrs[i]
#             if f_arr.shape[0]<862:
#                 new_f = np.zeros(862)
#                 for ii in range(f_arr.shape[0]):
#                     new_f[ii] = f_arr[ii]
#                 final.append(new_f)
#             else:
#                 final.append(f_arr[:862])
    
#     final = np.array(final).T
#     np.save(final_dataset+str(patient)+'.npy',final)
#     data_3d.append(final)

# data_3d = np.array(data_3d)
# df = {'features':data_3d}

zcr (862,)
rms (862,)
spectral_centroid (862,)
mfcc (24, 862)
chroma (12, 862)
spectral_contrast (7, 862)
spectral_slope (862,)


In [ ]:
# print(data_3d.shape)

(539, 862, 47)


## **Bi-LSTM**